In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Simple chain with LLMChain

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [3]:
chat = ChatOpenAI()

In [4]:
prompt_template = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [5]:
llm_chain = LLMChain(
    llm=chat,
    prompt=prompt_template
)

In [6]:
product = "AI Applications and AI Courses"

In [7]:
llm_chain.run(product)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'AI Mastery'

## Simple chain with LCEL (LangChain Expression Language)

In [8]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

In [9]:
prompt_template = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

In [10]:
new_llm_chain = prompt_template | chat | StrOutputParser()

In [11]:
new_llm_chain.invoke(
    {"product": product}
)

'1. AIverse Solutions\n2. AI Academy\n3. IntelliTech Solutions\n4. AI Innovators\n5. MindCraft AI\n6. AI Wizardry\n7. SynthAI Solutions\n8. AI Mastery\n9. CogniTech Academy\n10. AI Prodigy\n11. SmartAI Solutions\n12. AI Nexus\n13. AI Learners\n14. TechGenius AI\n15. AI Empowerment\n16. Brainwave AI\n17. AI Fusion\n18. InsightAI Solutions\n19. AI Mentors\n20. AI Pathfinders'

## Sequential Chain

In [12]:
from langchain.chains import SequentialChain

In [13]:
# prompt template 1: translate to Spanish
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to spanish:"
    "\n\n{Text}"
)

# chain 1: input= Text and output= Spanish_Translation
chain_one = LLMChain(
    llm=chat, 
    prompt=first_prompt, 
    output_key="Spanish_Translation"
)

In [14]:
# prompt template 2: summarize
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following text in 1 sentence:"
    "\n\n{Text}"
)

# chain 2: input= Text and output= summary
chain_two = LLMChain(
    llm=chat, 
    prompt=second_prompt, 
    output_key="summary"
)

In [15]:
# prompt template 3: identify language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following text:\n\n{Spanish_Translation}"
)

# chain 3: input= Spanish_Translation and output= language
chain_three = LLMChain(
    llm=chat, 
    prompt=third_prompt,
    output_key="language"
)

In [16]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=chat, 
    prompt=fourth_prompt,
    output_key="followup_message"
)

In [17]:
# overall_chain: input= Text 
# and output= Spanish_Translation,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Text"],
    output_variables=["Spanish_Translation", "summary","followup_message"],
    verbose=True
)

In [18]:
review = """
Being in a start-up myself, I read this book to find comfort 
and confirmation about the volatility and emotional roller-coaster 
that comes with a working at the brink of software-as-a service. 
This book was just what it promised - interviews from some of the 
great successes in SV and elsewhere, their humble and sometimes 
difficult beginnings ('against all odds') that I am experiencing 
right now. It's not a literary piece of work - never intended to 
be, on the contrary, I felt the writing style was just what fits 
with a life in the fast lane - little time, easy to read chapters, 
inspiring and thank god, very 'down to earth.'

The one critical point I would like to make: I am somewhat 
perplexed how the companies were chosen - there are so many 
other companies that could have fit the profile which seem 
much more of a success than some of the companies/products in 
the book (gmail? Comm'on, I guess the author wanted to have 
Google in there, but didn't get an interview so she went with 
gmail?). Other great companies are easy to find - they don't
even need to be in the consumer space. How about Salesforce.com? 
I definitely liked the mix of 'new' and 'experienced' start ups. 

This book was a breeze to read and insightful for us start-up 
enterpreneurs.
"""
overall_chain(review)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Text': "\nBeing in a start-up myself, I read this book to find comfort \nand confirmation about the volatility and emotional roller-coaster \nthat comes with a working at the brink of software-as-a service. \nThis book was just what it promised - interviews from some of the \ngreat successes in SV and elsewhere, their humble and sometimes \ndifficult beginnings ('against all odds') that I am experiencing \nright now. It's not a literary piece of work - never intended to \nbe, on the contrary, I felt the writing style was just what fits \nwith a life in the fast lane - little time, easy to read chapters, \ninspiring and thank god, very 'down to earth.'\n\nThe one critical point I would like to make: I am somewhat \nperplexed how the companies were chosen - there are so many \nother companies that could have fit the profile which seem \nmuch more of a success than some of the companies/products in \nthe book (gmail? Comm'on, I guess the author wanted to have \nGoogle in there, but didn

## Sequential Chain with LCEL (LangChain Expression Language)

*Option 1: with the simpler syntax we get just the output.*

In [19]:
from langchain.schema import StrOutputParser

In [20]:
new_sequence_chain = {"Spanish_Translation": first_prompt | chat | StrOutputParser()} | third_prompt | chat | StrOutputParser()

In [21]:
new_sequence_chain.invoke({"Text": review})

'The following text is in Spanish.'

*Option 2: with RunnablePassthrough we get input and output.*

In [22]:
from langchain.schema.runnable import RunnablePassthrough

In [23]:
# input: Text, output: Spanish_Translation
first_chain = first_prompt | chat | StrOutputParser()

In [24]:
# input: Text, output: summary
second_chain = second_prompt | chat | StrOutputParser()

In [25]:
# input: Spanish_Translation, output: language
third_chain = third_prompt | chat | StrOutputParser()

In [26]:
# input: summary, output: followup_message
fourth_chain = fourth_prompt | chat | StrOutputParser()

In [27]:
one_plus_three_sequence_chain = {"Spanish_Translation": first_chain} | RunnablePassthrough.assign(language=third_chain)

In [28]:
one_plus_three_sequence_chain.invoke({"Text": review})

{'Spanish_Translation': 'Siendo parte de una startup yo mismo, leí este libro para encontrar consuelo y confirmación acerca de la volatilidad y montaña rusa emocional que viene con el trabajo al borde del software como servicio. Este libro fue justo lo que prometía: entrevistas de algunos de los grandes éxitos en SV y otros lugares, sus humildes y a veces difíciles comienzos (\'contra todo pronóstico\') que estoy experimentando en este momento. No es una obra literaria, nunca pretendió serlo, al contrario, sentí que el estilo de escritura era justo lo que se ajusta a una vida en la vía rápida: poco tiempo, capítulos fáciles de leer, inspiradores y gracias a Dios, muy realistas.\n\nEl único punto crítico que me gustaría hacer es que me siento algo perplejo acerca de cómo se eligieron las empresas: hay tantas otras empresas que podrían haber encajado en el perfil y que parecen ser mucho más exitosas que algunas de las empresas/productos en el libro (¿gmail? Vamos, supongo que la autora q

## Router Chain

In [29]:
rock_template = """You are a very smart rock and roll professor. \
You are great at answering questions about rock and roll in a concise\
and easy to understand manner.

Here is a question:
{input}"""


politics_template = """You are a very good politics professor. \
You are great at answering politics questions..

Here is a question:
{input}"""


history_template = """You are a very good history teacher. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.

Here is a question:
{input}"""


sports_template = """ You are a sports teacher.\
You are great at answering sports questions.

Here is a question:
{input}"""

In [30]:
prompt_infos = [
    {
        "name": "rock", 
        "description": "Good for answering questions about rock and roll", 
        "prompt_template": rock_template
    },
    {
        "name": "politics", 
        "description": "Good for answering questions about politics", 
        "prompt_template": politics_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "sports", 
        "description": "Good for answering questions about sports", 
        "prompt_template": sports_template
    }
]

In [31]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [32]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=chat, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat, prompt=default_prompt)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.

REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [35]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(
    llm=chat, 
    prompt=router_prompt
)

In [36]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [37]:
chain.run("Who was Joe DiMaggio? Respond in less than 100 words")



> Entering new MultiPromptChain chain...


/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


sports: {'input': 'Who was Joe DiMaggio?'}
> Finished chain.


"Joe DiMaggio was a legendary American baseball player who played for the New York Yankees. He is considered one of the greatest baseball players of all time. He played as an outfielder and had an illustrious career from 1936 to 1951, during which he won numerous accolades and set several records. DiMaggio is best known for his 56-game hitting streak in 1941, a record that still stands today. He was a 13-time All-Star, a nine-time World Series champion, and a three-time Most Valuable Player (MVP) award winner. DiMaggio's skills, grace, and consistency made him a beloved figure in the world of baseball."

In [38]:
chain.run("What country won the last soccer world cup?")



> Entering new MultiPromptChain chain...


/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


sports: {'input': 'What country won the last soccer world cup?'}
> Finished chain.


'The last soccer World Cup was won by France. They emerged as the champions in the 2018 FIFA World Cup held in Russia.'

## Router Chain with LCEL (LangChain Expression Language)

In [39]:
from langchain.prompts import PromptTemplate

rock_template = """You are a very smart rock and roll professor. \
You are great at answering questions about rock and roll in a concise\
and easy to understand manner.

Here is a question:
{input}"""

rock_prompt = PromptTemplate.from_template(rock_template)

politics_template = """You are a very good politics professor. \
You are great at answering politics questions..

Here is a question:
{input}"""

politics_prompt = PromptTemplate.from_template(politics_template)

history_template = """You are a very good history teacher. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.

Here is a question:
{input}"""

history_prompt = PromptTemplate.from_template(history_template)

sports_template = """ You are a sports teacher.\
You are great at answering sports questions.

Here is a question:
{input}"""

sports_prompt = PromptTemplate.from_template(sports_template)

In [40]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

In [41]:
general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)
prompt_branch = RunnableBranch(
  (lambda x: x["topic"] == "rock", rock_prompt),
  (lambda x: x["topic"] == "politics", politics_prompt),
  (lambda x: x["topic"] == "history", history_prompt),
  (lambda x: x["topic"] == "sports", sports_prompt),
  general_prompt
)

In [42]:
from typing import Literal

from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function


class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    
    topic: Literal["rock", "politics", "history", "sports"]
    "The topic of the user question. One of 'rock', 'politics', 'history', 'sports' or 'general'."


classifier_function = convert_pydantic_to_openai_function(TopicClassifier)
llm = ChatOpenAI().bind(functions=[classifier_function], function_call={"name": "TopicClassifier"}) 
parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")
classifier_chain = llm | parser

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [43]:
from operator import itemgetter

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain) 
    | prompt_branch 
    | ChatOpenAI()
    | StrOutputParser()
)

In [44]:
final_chain.invoke(
    {"input": "Who was Napoleon Bonaparte?"}
)

"Thank you for the compliment! Napoleon Bonaparte was a prominent figure in history, known for his military genius and his impact on Europe during the late 18th and early 19th centuries. He was born on August 15, 1769, in Corsica, which was a French territory at the time. Napoleon rose to power during the French Revolution and eventually became the Emperor of France.\n\nNapoleon's military career began when he joined the French Army during the revolutionary years. He quickly rose through the ranks due to his tactical abilities and leadership qualities. In 1799, he staged a coup d'état and became the First Consul of France. In 1804, he declared himself Emperor Napoleon I.\n\nDuring his reign, Napoleon implemented several significant reforms in France, known as the Napoleonic Code. This legal system sought to establish equality before the law, religious toleration, and the protection of property rights. He also introduced various administrative, educational, and economic reforms that mod